In [1]:
#read yaml file
import yaml

file_name = "cem_test.yaml"

with open(f"configs/{file_name}", 'r') as stream:
    try:
        config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)
print(config)


{'solver': {'type': 'vqa', 'pqc': {'type': 'wfqaoa', 'layers': 5, 'penalty': 3}, 'params_inits': {'angles': [[0.5, 0.5, 0.5, 0.5, 0.5], [1, 1, 1, 1, 1]], 'hyper_args': [1, 2.5, 2.5]}, 'hyper_optimizer': {'type': 'cem', 'processes': 5, 'samples_per_epoch': 'PLACEHOLDER', 'epochs': 'PLACEHOLDER', 'elite_frac': 'PLACEHOLDER', 'bounds': [[1, 10], [1, 10], [1, 10]], 'disable_tqdm': True}}, 'problem': {'type': 'knapsack', 'max_weight': 2, 'items': [[1, 2], [1, 2], [1, 1]]}}


In [2]:
samples_per_epochs = [100, 500, 1000, 2000]
elite_fracs = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3]


In [3]:
from QHyper.solvers import solver_from_config
from QHyper.util import weighted_avg_evaluation
import copy

def run_experiment(samples_per_epoch, elite_frac):
    _config = copy.deepcopy(config)
    _config['solver']['hyper_optimizer']['samples_per_epoch'] = samples_per_epoch
    _config['solver']['hyper_optimizer']['elite_frac'] = elite_frac
    _config['solver']['hyper_optimizer']['epochs'] = 10_000 // samples_per_epoch
    solver = solver_from_config(_config)
    results = solver.solve()

    return weighted_avg_evaluation(results.results_probabilities, solver.problem.get_score)


In [4]:
import numpy as np

for samples_per_epoch in samples_per_epochs:
    for elite_frac in elite_fracs:
        print(f"Running experiment with samples_per_epoch={samples_per_epoch}, elite_frac={elite_frac}")

        results = [
            run_experiment(samples_per_epoch, elite_frac)
            for _ in range(10)
        ]
        print(f"Results: {np.mean(results)} +- {np.std(results)}")

        with open(f"results/{file_name}", 'a') as f:
            f.write(f"{samples_per_epoch},{elite_frac},{np.mean(results)},{np.std(results)}\n")


Running experiment with samples_per_epoch=100, elite_frac=0.05


KeyboardInterrupt: 